# Capstone: Exploratory Prediction Modeling

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# import project utils
import sys
sys.path.append('../src')

import data_utils
from data_utils import Config

import graph_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, log_loss

from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.INFO)

## The Data: San Francisco Police Department Incident Reports

### Read the Data

In [6]:
# Which dataset to work from?

sample_file = data_utils.select_sample_csv_file(pct=10)
print(f'Selected sample file: {sample_file}')

Selected sample file: ../data/incidents_clean_10_pct.csv


In [7]:
current_raw_df, current_clean_df = data_utils.get_clean_data_from_csv(sample_file)

Reading file: ../data/incidents_clean_10_pct.csv ... Done: 88,717 rows, 37 columns
... Converting datetime to timeseries ... Done
... Setting index to datetime ... Done
Done


In [8]:
data = data_utils.preprocess_data(current_raw_df.copy())

Pre-processing ... 
... Dropping unwanted columns ... 
... preprocess_drop_cols: Column Unnamed: 0 dropped
... preprocess_drop_cols: Column esncag_-_boundary_file dropped
... preprocess_drop_cols: Column central_market/tenderloin_boundary_polygon_-_updated dropped
... preprocess_drop_cols: Column civic_center_harm_reduction_project_boundary dropped
... preprocess_drop_cols: Column hsoc_zones_as_of_2018-06-05 dropped
... preprocess_drop_cols: Column invest_in_neighborhoods_(iin)_areas dropped
... preprocess_drop_cols: Column report_type_code dropped
... preprocess_drop_cols: Column report_type_description dropped
... preprocess_drop_cols: Column filed_online dropped
... preprocess_drop_cols: Column intersection dropped
... preprocess_drop_cols: Column cnn dropped
... preprocess_drop_cols: Column point dropped
... preprocess_drop_cols: Column supervisor_district dropped
... preprocess_drop_cols: Column supervisor_district_2012 dropped
... preprocess_drop_cols: Column current_supervisor_d

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82242 entries, 2022-02-10 07:59:00 to 2021-07-09 00:22:00
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             82242 non-null  object 
 1   time             82242 non-null  object 
 2   year             82242 non-null  int64  
 3   day_of_week      82242 non-null  object 
 4   category         82242 non-null  object 
 5   resolution       82242 non-null  object 
 6   police_district  82242 non-null  object 
 7   neighborhood     82242 non-null  object 
 8   latitude         82242 non-null  float64
 9   longitude        82242 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 6.9+ MB


## Summary of EDA

After cleaning the data and performing basic EDA, we have established the following:

1. Target variable `category`
   * Evenly spread across time
   * Incidence of crimes is extremely skewed/unbalanced by category. Larceny (29.02%) by far outweighing the other top-10 categories with each being in the single digits
3. Features impacting `category`
   * Affected by incident time and date components: date, time, day of week, month, year, etc
   * Affected by police disctrict
   * Affect by latitude and logitude (TODO: need visualization)
4. We artificially removed nulls (TODO: will come back to impute data later)

## Feature Engineering

In [13]:
data.head(2)

,date,time,year,day_of_week,category,resolution,police_district,neighborhood,latitude,longitude
datetime,,,,,,,,,,
2022-02-10 07:59:00,2022/02/10,07:59,2022,Thursday,Recovered Vehicle,Open or Active,Ingleside,West of Twin Peaks,37.728975,-122.468077
2022-11-17 23:30:00,2022/11/17,23:30,2022,Thursday,Missing Person,Open or Active,Mission,Mission,37.762579,-122.421662


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82242 entries, 2022-02-10 07:59:00 to 2021-07-09 00:22:00
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             82242 non-null  object 
 1   time             82242 non-null  object 
 2   year             82242 non-null  int64  
 3   day_of_week      82242 non-null  object 
 4   category         82242 non-null  object 
 5   resolution       82242 non-null  object 
 6   police_district  82242 non-null  object 
 7   neighborhood     82242 non-null  object 
 8   latitude         82242 non-null  float64
 9   longitude        82242 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 6.9+ MB


### Encoding: Time-based columns

Let's unpack the date and time into their components that are still missing so there is less to encode:

In [17]:
data['hour'] = data.index.map(lambda x: x.hour)
data['minute'] = data.index.map(lambda x: x.minute)
data['day'] = data.index.map(lambda x: x.day)
data['month'] = data.index.map(lambda x: x.month)

Now let's encode day_of_week to numeric values:

In [19]:
enc_dow = LabelEncoder()
enc_dow.fit(data.day_of_week.unique())
data['dow'] = enc_dow.transform(data.day_of_week)

Let's mark the redundant columns to be dropped after feature engineering:

In [21]:
drop_encoded_cols = ['date', 'time', 'day_of_week']

### Encoding: Resolution

We will also drop the resolution column since it doesn't impact crime prediction:

In [24]:
data.resolution.value_counts()

resolution
Open or Active          65847
Cite or Arrest Adult    16395
Name: count, dtype: int64

In [25]:
drop_encoded_cols.append('resolution')

### Encoding: Category

In [27]:
enc_cat = LabelEncoder()
enc_cat.fit(data.category.unique())
data.category = enc_cat.transform(data.category)

### Encoding: Police District

In [29]:
enc_pd = LabelEncoder()
enc_pd.fit(data.police_district.unique())
data['pd'] = enc_pd.transform(data.police_district)

### Encoding: Neighborhood

In [31]:
enc_hood = LabelEncoder()
enc_hood.fit(data.neighborhood.unique())
data.neighborhood = enc_hood.transform(data.neighborhood)

### Dropping Redundant Columns

We can now drop the encoded columns:

In [34]:
drop_encoded_cols.append('police_district')

print(f'Dropping encoded columns: {drop_encoded_cols}')
data.drop(columns=drop_encoded_cols, inplace=True)

Dropping encoded columns: ['date', 'time', 'day_of_week', 'resolution', 'police_district']


In [35]:
data.head(2)

,year,category,neighborhood,latitude,longitude,hour,minute,day,month,dow,pd
datetime,,,,,,,,,,,
2022-02-10 07:59:00,2022,33,39,37.728975,-122.468077,7,59,10,2,4,2
2022-11-17 23:30:00,2022,23,18,37.762579,-122.421662,23,30,17,11,4,3


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82242 entries, 2022-02-10 07:59:00 to 2021-07-09 00:22:00
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          82242 non-null  int64  
 1   category      82242 non-null  int64  
 2   neighborhood  82242 non-null  int64  
 3   latitude      82242 non-null  float64
 4   longitude     82242 non-null  float64
 5   hour          82242 non-null  int64  
 6   minute        82242 non-null  int64  
 7   day           82242 non-null  int64  
 8   month         82242 non-null  int64  
 9   dow           82242 non-null  int64  
 10  pd            82242 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 7.5 MB


## Data Preparation

### Create Train/Test Splits

In [39]:
X = data.drop('category', axis='columns')
y = data['category']

In [40]:
# OneHot Encode the features and drop the first value to reduce multicollinearity
X = pd.get_dummies(X, drop_first=True)

In [41]:
# Consistent random_state for the project
print(f'Project-wide random_state: {Config.RANDOM_STATE}')

Project-wide random_state: 42


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=Config.RANDOM_STATE)

In [43]:
# spot-check feature encoding
X.T.iloc[:, 0:5]

datetime,2022-02-10 07:59:00,2022-11-17 23:30:00,2023-12-23 22:50:00,2018-09-05 13:20:00,2022-01-22 10:28:00
year,2022.000000,2022.000000,2023.000000,2018.000000,2022.000000
neighborhood,39.000000,18.000000,22.000000,35.000000,33.000000
latitude,37.728975,37.762579,37.802755,37.779992,37.783288
longitude,-122.468077,-122.421662,-122.413623,-122.413487,-122.408952
hour,7.000000,23.000000,22.000000,13.000000,10.000000
minute,59.000000,30.000000,50.000000,20.000000,28.000000
day,10.000000,17.000000,23.000000,5.000000,22.000000
month,2.000000,11.000000,12.000000,9.000000,1.000000
dow,4.000000,4.000000,2.000000,6.000000,2.000000
pd,2.000000,3.000000,1.000000,9.000000,9.000000


### Feature Scaling

In [45]:
# Scale the data - we'll use StandardScaler for the baseline model
logging.debug('Scaling data')
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

## Model Exploration

We will now evaluate different models for predicting the Crime Category from our features:

In [48]:
X_train.columns

Index(['year', 'neighborhood', 'latitude', 'longitude', 'hour', 'minute',
       'day', 'month', 'dow', 'pd'],
      dtype='object')

We will evaluate the following models:

* Logistic Regression with L1 Regularization
* K Nearest Neighbors
* Stochastic Gradient Descent

### Evaluation Metrics

In this project, we are predicting or classifyig across 49 crime categories. We will use two evaluation metrics to compare our models:

1. **Accuracy**: Measures the proportion of correct predictions over all predictions made. The accuracy benchmark is 1/49 or 2.04% given our crime categories
2. **Log_Loss**: Measures the accuracy of a classifier by penalizing false classifications. It does this by taking the negative logarithm of the predicted probability for the true class. The goal is to minimize this loss, meaning that higher probabilities are assigned to the correct classes
   * TODO: Benchmark???

While accuracy provides a simple measure of correctness, log-loss offers a more nuanced view by considering how confident those predictions are. We'll use them together for a comprehensive evaluation and to learn more about them

### Baseline DummyClassifier

In [53]:
results = []

In [54]:
base = DummyClassifier(strategy='uniform')
base.fit(X_train_scaled, y_train)
y_preds = base.predict(X_test)
pred_probs = base.predict_proba(X_test)
base_acc = accuracy_score(y_test, y_preds)
base_loss = log_loss(y_test, pred_probs)

label='Baseline: DummyClassifier - strategy=uniform'
results.append([label, base_acc, base_loss])
print(f'{label}: accuracy: {base_acc}, log_loss: {base_loss}')

Baseline: DummyClassifier - strategy=uniform: accuracy: 0.01981883397167001, log_loss: 3.8501476017100575


### LogisticRegresson (L1)

In [56]:
lr = LogisticRegression(penalty='l1', solver='saga',
                        max_iter=1000, verbose=1, n_jobs=3, random_state=Config.RANDOM_STATE)

lr.fit(X_train_scaled, y_train)
y_preds = lr.predict(X_test)
pred_probs = lr.predict_proba(X_test)
lr_acc = accuracy_score(y_test, y_preds)
lr_loss = log_loss(y_test, pred_probs)

label='LogisticRegression (L1)'
results.append([label, lr_acc, lr_loss])
print(f'{label}: accuracy: {lr_acc}, log_loss: {lr_loss}')

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


convergence after 163 epochs took 45 seconds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


LogisticRegression (L1): accuracy: 0.00024317587695300627, log_loss: 36.03488844209566


### K-Nearest Neighbors

In [58]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)
y_preds = knn.predict(X_test)
pred_probs = knn.predict_proba(X_test)
knn_acc = accuracy_score(y_test, y_preds)
knn_loss = log_loss(y_test, pred_probs)

label='K-Nearest Neighbors'
results.append([label, knn_acc, knn_loss])
print(f'{label}: accuracy: {knn_acc}, log_loss: {knn_loss}')

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


K-Nearest Neighbors: accuracy: 0.22548483190467505, log_loss: 34.182305229585396


### Random Forest Ensemble

In [60]:
rf = RandomForestClassifier(n_estimators=500, max_depth=15,
                            min_samples_leaf=5, min_samples_split=25, 
                            random_state=Config.RANDOM_STATE, verbose=1, n_jobs=2)
rf.fit(X_train_scaled, y_train)
y_preds = rf.predict(X_test)
pred_probs = rf.predict_proba(X_test)
rf_acc = accuracy_score(y_test, y_preds)
rf_loss = log_loss(y_test, pred_probs)

label='Random Forest'
results.append([label, rf_acc, rf_loss])
print(f'{label}: accuracy: {rf_acc}, log_loss: {rf_loss}')

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.9s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   12.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:   26.4s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:   31.8s finished
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.6s finished
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names

Random Forest: accuracy: 0.2926013739437048, log_loss: 16.45791388036314


[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.6s finished


### XGBoost Ensemble

In [62]:
# xgb_clf = XGBClassifier(n_estimators=500, objective="multi:softprob", 
#                         verbose=1, n_jobs=2, random_state=Config.RANDOM_STATE)
# xgb_clf.fit(X_train, y_train)
# y_preds = xgb_clf.predict(X_test)
# pred_probs = xgb_clf.predict_proba(X_test)
# xgb_acc = accuracy_score(y_test, y_preds)
# xgb_loss = log_loss(y_test, pred_probs)

# label='XGBoost'
# results.append([label, xgb_acc, xgb_loss])
# print(f'{label}: accuracy: {xgb_acc}, log_loss: {xgb_loss}')

In [63]:
results_df = pd.DataFrame(results,
                          columns=['Label', 'Accuracy', 'Log_Loss']
                         ).set_index('Label')
results_df

,Accuracy,Log_Loss
Label,,
Baseline: DummyClassifier - strategy=uniform,0.019819,3.850148
LogisticRegression (L1),0.000243,36.034888
K-Nearest Neighbors,0.225485,34.182305
Random Forest,0.292601,16.457914
